In [1]:
import bs4
import urllib3
import csv

In [2]:
def scrape_price(code):
    target = "http://finance.naver.com/item/coinfo.nhn?code=" + code + "#"
    pm = urllib3.PoolManager()
    html = pm.urlopen(url=target, method="GET").data
    soup = bs4.BeautifulSoup(html, 'lxml')
    data_list = soup.find_all("dl", class_ = "blind")[0].find_all("dd")
    data_dic = {}
    first_line = data_list[3].text.split()
    data_dic["price"] = first_line[1]
    if first_line[5] == "마이너스":
        data_dic["value_dif"] = "-" + first_line[4]
        data_dic["value_dif_per"] = "-" + first_line[6]
    else:
        data_dic["value_dif"] = first_line[4]
        data_dic["value_dif_per"] = first_line[6]
    var_list = ["yestday_close", "today_open", "today_high", "highest_poss", "today_low", \
                "lowest_poss", "trade_vol", "trade_amount"]
    num = 4
    for var in var_list:
        data_dic[var] = data_list[num].text.split()[1]
        num += 1
    
    return data_dic    

In [69]:
def add_price_to_file(filename, code):
    
    fieldnames = ["price", "value_dif", "value_dif_per", "yestday_close", \
                  "today_open", "today_high", "highest_poss", "today_low", \
                  "lowest_poss", "trade_vol", "trade_amount"]
    row = scrape_price(code)
    with open(filename, "w") as f:
        writer = csv.DictWriter(f, fieldnames = fieldnames)
        writer.writeheader()
        writer.writerow(row)
        
    return